In [1]:
import numpy as np
import pandas as pd

from sklearn.impute import (SimpleImputer,KNNImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import lightgbm as lgb

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [2]:
def fill_missing_values(X, n_neighbors = 75, method="KNN"): 
    
    # normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    X_norma = (X-X_ave)/X_std
    
    # use KNNImputer
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights = 'distance') if method=="KNN"\
        else SimpleImputer(missing_values=np.nan, strategy='median')
    
    X_norma_fixed = imputer.fit_transform(X_norma)
    
    return X_norma_fixed

# def remove_outliers(X, y):
#     print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
#     iforest = IsolationForest(max_samples=200, random_state=1, contamination='auto')
#     iforest.fit(X)
#     iforest_outlier_pred = iforest.predict(X)

#     mask = (iforest_outlier_pred!=-1)
#     X , y = X[mask, :], y[mask]
#     print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
#     return X, y

def remove_outliers(X, y):
    print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    Z = np.c_[X, y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
    X , y = X[mask, :], y[mask]
    print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
    return X, y

def select_features(X, y, X_test, feature_num=50):
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=80, random_state=1)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    return X, X_test

In [3]:
X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

In [4]:
X_train_missing_indices = X_train[X_train==np]
X_train = fill_missing_values(X_train, n_neighbors=75)
X_test = fill_missing_values(X_test)
print(X_train.shape)
print(X_test.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


(1212, 828)
(776, 828)


In [5]:
X_train,y_train = remove_outliers(X_train,y_train)

IsolationForest-Traing data shape before removed: (1212, 828)
IsolationForest-Traing data shape after removed: (1200, 828)


In [6]:
# X, X_test = feature_reduction(X, X_test,750)
X_train, X_test = select_features(X_train, y_train, X_test,feature_num = 50)
print("Traing data shape after selection: {}".format(X_train.shape))
print("Testing data shape after selection: {}".format(X_test.shape))

Traing data shape after selection: (1200, 50)
Testing data shape after selection: (776, 50)


In [7]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True

def fit_model_and_pred(X_train, y_train, X_val, y_val, X_test, alpha, n_restarts_optimizer, random_state):
    kernel = RBF(0.5, (1e-4, 10))
    gpr = GaussianProcessRegressor(kernel=kernel, random_state=random_state, n_restarts_optimizer=n_restarts_optimizer, alpha=alpha, normalize_y=True)
    gpr.fit(X_train, y_train)
    score = gpr.score(X_val, y_val)   
    y_pred = gpr.predict(X_test) 

    return score, y_pred

def train_k_fold(X, y, alpha, n_restarts_optimizer, random_state, fold_num=10):
    kf = KFold(n_splits=fold_num, shuffle=False)
    kf.get_n_splits(X)
    test_score = 0.0
    
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_val, 
                                           alpha, n_restarts_optimizer, random_state)

        #print('The obtained validation r2 score is : ',score)
        test_score += score
    return test_score/fold_num
    

In [8]:
alpha_list = [1e-2, 0.25e-1, 0.5e-1, 0.6e-1, 0.7e-1, 0.8e-1, 0.9e-1, 1e-1, 2.5e-1, 5e-1]
n_restarts_optimizer_list = [1,5,10]
random_state_list = [0,1,2]

max_score = -10000
best_alpha = 0.1
best_n_restarts_optimizer = 10
best_random_state = 0

for alpha in tqdm(alpha_list):
    for n_restarts_optimizer in n_restarts_optimizer_list:
        for random_state in random_state_list:
            score = train_k_fold(X_train, y_train, alpha, n_restarts_optimizer, random_state, fold_num=10) 
            
            if(score > max_score):
                max_score = score
                best_alpha = alpha
                best_n_restarts_optimizer = n_restarts_optimizer
                best_random_state = random_state

 40%|████      | 4/10 [09:03<13:35, 135.95s/it]/opt/anaconda3/lib/python3.7/site-packages/sklearn/gaussian_process/_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/gaussian_process/_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
100%|██████████| 10/10 [21:48<00:00, 130.85s/it]


In [9]:
print(max_score)
print(best_alpha)
print(best_n_restarts_optimizer)
print(best_random_state)

0.6655601883328591
0.07
1
0
